# What makes Happiness?

For a long time, people have been interested in what makes us happy and how we can improve well-being in society. One big question is whether having more money or wealth actually makes people happier, or if there are other things that matter more. We often hear the saying “money can’t buy happiness,” but the real answer is a bit more complicated than that.

In this data story, we will explore how happiness relates to different economic and social factors using information from the World Happiness Report 2019 and World Development Indicators. This report looks at how happy people are in different countries and compares that with demographic variables like GDP per capita, Gross National Income (GNI), the gini index, unemployment rates, education levels, and life expectancy.

We want to find out if richer countries really have happier people, and if so, how strong this connection is. But we will also look beyond money to see how things like having a job, going to school, and living a long, healthy life affect happiness. For example, being unemployed might make people less happy even if their country is wealthy, and having a good education could improve well-being in ways that money alone can’t.

First, we will compare happiness scores with income indicators like GDP per capita and GNI. Then, we will analyze how unemployment rates relate to happiness. After that, we will look at the gini index, education and life expectancy to see how these factors could affect happiness scores across countries.

By comparing these different aspects, we hope to better understand what really contributes to happiness around the world. This will help us see whether the saying “money buys happiness” really holds. And if it doesn’t hold, we could find out what does make happiness.


It is often proposed that greater national wealth leads to higher levels of happiness among citizens. To see wether they are correlated, we analysed how average happiness scores relate to three key economic indicators: Gross Domestic Product (GDP), Gross National Income (GNI), and the Gini Index (Income inequality).

The trendlines suggest that countries with higher GDP and GNI often report higher happiness levels. This supports the general belief that wealth leads to happiness due to the fact that wealth increases access to basic needs, and social stability, factors that are positively associated with well-being and happiness. However, the relationship is not linear, and several wealthy countries show only moderate happiness scores, this suggests other factors are still extremely relevant and happiness cannot be predicted by looking at wealth alone. Correlation does not imply causation, and the observed pattern may be influenced by unmeasured factors.

The third plot, Happiness vs Gini Index, presents a different insight. Within this dataset, a negative correlation is visible, with happiness tending to decline as income inequality (measured by the Gini Index) increases. . In other words, even among wealthier nations, greater inequality is associated with lower average happiness. This aligns with findings by researchers such as Oishi et al., (2011), who in their research found that americans reported being happier on average in years where the national income inequality was less. This suggests income inequality might be a factor which has impact on the happiness . However income inequality may be acting as a substitute for how people perceive the fairness in society and the level of social mobility people have. Happiness is also a subjective measurement, and cultural differences in how inequality is perceived can vary widely, potentially affecting how people respond in surveys.

The plots suggest that while wealth does matter, its distribution may be even more of a factor. Factors such as income equality are likely important to how people will respond to questions about their happiness. 

In [10]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from sklearn.linear_model import LinearRegression

init_notebook_mode(connected=True)

df = pd.read_csv('final.csv')

df['Happiness_rank'] = df['Happiness'].rank(method='min', ascending=False).astype(int)
total_countries = df['Country'].nunique()

df_gdp  = df.dropna(subset=['GDP_current_usd', 'Happiness', 'Country', 'Happiness_rank'])
df_gni  = df.dropna(subset=['GNI_current_usd', 'Happiness', 'Country', 'Happiness_rank'])
df_gini = df.dropna(subset=['Gini_index',       'Happiness', 'Country', 'Happiness_rank'])

def add_trend_line(fig, x, y, name, color, visible=True):
    
    x_vals = np.log(x) if name in ['GDP', 'GNI'] else x
    x_vals = x_vals.values.reshape(-1, 1)
    y_vals = y.values.reshape(-1, 1)
    
    model = LinearRegression()
    model.fit(x_vals, y_vals)
    
    
    x_trend = np.linspace(min(x), max(x), 100)
    x_trend_input = np.log(x_trend) if name in ['GDP', 'GNI'] else x_trend
    
    y_trend = model.predict(x_trend_input.reshape(-1, 1)).flatten()
    
    fig.add_trace(
        go.Scatter(
            x=x_trend,
            y=y_trend,
            mode='lines',
            name=f'{name} Trend',
            line=dict(color=color, dash='dash'),
            visible=visible,
            hoverinfo='skip'
        )
    )

trace_gdp = go.Scatter(
    x=df_gdp['GDP_current_usd'],
    y=df_gdp['Happiness'],
    mode='markers',
    name='GDP (US$)',
    visible=True,
    text=df_gdp['Country'],
    customdata=df_gdp['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "GDP (US$): %{x:$,.0f}<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trace_gni = go.Scatter(
    x=df_gni['GNI_current_usd'],
    y=df_gni['Happiness'],
    mode='markers',
    name='GNI (US$)',
    visible=False,
    text=df_gni['Country'],
    customdata=df_gni['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "GNI (US$): %{x:$,.0f}<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trace_gini = go.Scatter(
    x=df_gini['Gini_index'],
    y=df_gini['Happiness'],
    mode='markers',
    name='Gini Index',
    visible=False,
    text=df_gini['Country'],
    customdata=df_gini['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "Gini Index: %{x:.2f}<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

fig = go.Figure(data=[trace_gdp, trace_gni, trace_gini])

add_trend_line(fig, df_gdp['GDP_current_usd'], df_gdp['Happiness'], 'GDP', 'blue', True)
add_trend_line(fig, df_gni['GNI_current_usd'], df_gni['Happiness'], 'GNI', 'blue', False)
add_trend_line(fig, df_gini['Gini_index'], df_gini['Happiness'], 'Gini', 'blue', False)

steps = [
    dict(
        method='update',
        label='GDP (US$)',
        args=[
            {'visible': [True, False, False]},
            {
                'title': 'Happiness vs GDP (current US$)',
                'xaxis': {'title': 'GDP (current US$)', 'type': 'log'},
                'yaxis': {'title': 'Happiness'}
            }
        ]
    ),
    dict(
        method='update',
        label='GNI (US$)',
        args=[
            {'visible': [False, True, False]},
            {
                'title': 'Happiness vs GNI (current US$)',
                'xaxis': {'title': 'GNI (current US$)', 'type': 'log'},
                'yaxis': {'title': 'Happiness'}
            }
        ]
    ),
    dict(
        method='update',
        label='Gini Index',
        args=[
            {'visible': [False, False, True]},
            {
                'title': 'Happiness vs Gini Index',
                'xaxis': {'title': 'Gini Index', 'type': 'linear'},
                'yaxis': {'title': 'Happiness'}
            }
        ]
    )
]

fig.update_layout(
    sliders=[dict(
        active=0,
        currentvalue={'prefix': 'Metric: '},
        pad={'t': 50},
        steps=steps
    )],
    title="Happiness vs GDP (current US$)",
    width=900,
    height=600,
    margin={'t':80},
    xaxis=dict(
        title='GDP (current US$)',
        type='log'
    ),
    yaxis=dict(
        title='Happiness'
    )
)

iplot(fig)


In [8]:
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

df = pd.read_csv('final.csv')

df['Happiness_rank'] = df['Happiness'].rank(method='min', ascending=False).astype(int)
total_countries = df['Country'].nunique()

df_edu   = df.dropna(subset=['Edu_expenditure_pct', 'Happiness', 'Country', 'Happiness_rank'])
df_unemp = df.dropna(subset=['Unemployment_rate',    'Happiness', 'Country', 'Happiness_rank'])
df_le    = df.dropna(subset=['Life_expectancy',      'Happiness', 'Country', 'Happiness_rank'])

trace_edu = go.Scatter(
    x=df_edu['Edu_expenditure_pct'],
    y=df_edu['Happiness'],
    mode='markers',
    name='Education Expenditure',
    visible=True,
    text=df_edu['Country'],
    customdata=df_edu['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "Education Expenditure: %{x:.1f}% of GDP<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trace_unemp = go.Scatter(
    x=df_unemp['Unemployment_rate'],
    y=df_unemp['Happiness'],
    mode='markers',
    name='Unemployment Rate',
    visible=False,
    text=df_unemp['Country'],
    customdata=df_unemp['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "Unemployment Rate: %{x:.1f}%<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trace_le = go.Scatter(
    x=df_le['Life_expectancy'],
    y=df_le['Happiness'],
    mode='markers',
    name='Life Expectancy',
    visible=False,
    text=df_le['Country'],
    customdata=df_le['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "Life Expectancy: %{x:.1f} years<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

fig = go.Figure(data=[trace_edu, trace_unemp, trace_le])

steps = [
    dict(
        method='update',
        label='Education Expenditure',
        args=[
            {'visible': [True, False, False]},
            {
                'title': 'Happiness vs Education Expenditure',
                'xaxis': {'title': 'Education Expenditure (% of GDP)', 'type': 'linear'},
                'yaxis': {'title': 'Happiness'}
            }
        ]
    ),
    dict(
        method='update',
        label='Unemployment Rate',
        args=[
            {'visible': [False, True, False]},
            {
                'title': 'Happiness vs Unemployment Rate',
                'xaxis': {'title': 'Unemployment Rate (%)', 'type': 'linear'},
                'yaxis': {'title': 'Happiness'}
            }
        ]
    ),
    dict(
        method='update',
        label='Life Expectancy',
        args=[
            {'visible': [False, False, True]},
            {
                'title': 'Happiness vs Life Expectancy',
                'xaxis': {'title': 'Life Expectancy (years)', 'type': 'linear'},
                'yaxis': {'title': 'Happiness'}
            }
        ]
    )
]

fig.update_layout(
    sliders=[dict(
        active=0,
        currentvalue={'prefix': 'Metric: '},
        pad={'t': 50},
        steps=steps
    )],
    title="Happiness vs Education Expenditure",
    width=900,
    height=600,
    margin={'t':80}
)

iplot(fig)


In [2]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('final.csv')

fig = px.choropleth(
    df,
    locations='Country',
    locationmode='country names',
    color='Happiness',
    hover_name='Country',
    color_continuous_scale='Plasma',
    title='World Happiness Score by Country',
    width=1200,
    height=600
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    margin={"r":0,"t":60,"l":0,"b":0}
)

fig.show()
